In [1]:
import Bio
from Bio import SeqIO
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import itertools
import pandas as pd 
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from itertools import product
import random
from six.moves import cPickle as pickle
import xgboost as xgb
from Bio.SeqIO.FastaIO import SimpleFastaParser
from joblib import Parallel, delayed
import gc
from multiprocessing.dummy import Pool as ThreadPool

# data_root = "C:\\Users\\Tigran PC\\Desktop\\MicrobiomicsData\\data\\" # notebook
# data_root = "/Users/tigran/Desktop/sbv/data/" # imac
data_root = "C:\\Users\\Administrator\\Repos\\Microbiomics\\data\\" # server

sub_data_root = data_root + "sample01split/"

In [2]:
def make_sub_seq_bank(initial_string, sub_seq_len):
    return [''.join(tup) for tup in  list(set(product(set(initial_string), repeat = sub_seq_len)))]


In [3]:
s1 = 'GACGAGTCGGCGGCGGCCGCGGCCGCCGCCTGCTTCGAGCAGGCCGCACAAATGTGGGCCGACGAGCGCGATGCGATCGATGCGCTGCTGCGCGCCGCGCAGCCGGCGCTCAACCAGCGCTCGCACAAGCCCGAGGCGATCGCCGATGCGT'
s2 = 'GCGTGAACCCGAGCTATTCGCCGCCGCAGGTGATCCGCGGGCTTGCCGCCCGCTTGCCCGACGAGCGCCGCTGGGCCGCGCTGATGACGAGCACCGGCCGCGTGCTGCTCGACACCGCACCGAAGGGCTTCGCGCCGGACTGGGCGCTGTA' 
seq_list = []
sub_seq_bank = make_sub_seq_bank('ACTG', 4)

for i in range(1000):
    seq_list.append(s1)
    seq_list.append(s2)
    

# def freq(seq):
#     map_output= map(lambda x : seq.count(x), sub_seq_bank)
#     sub_seq_count = list(map_output)
#     return sub_seq_count

# def freq1(seq, lable):
#     sub_seq_count = []
#     for sub_seq in sub_seq_bank:
#         sub_seq_count.append(seq.count(sub_seq))
#     sub_seq_count.append(lable)
#     return sub_seq_count


def make_f(seq_list):
    data_list = []
    for i, seq in enumerate(seq_list):
        sub_seq_count = list(map(seq.count, sub_seq_bank))
        sub_seq_count.append("A"*(i+1))
        data_list.append(sub_seq_count)
    return data_list
    

In [4]:
%time d = make_f(seq_list)

Wall time: 548 ms


In [5]:
# %time results = Parallel(n_jobs=1,  backend="threading")(map(delayed(freq1), seq_list, 20000*["A"]))

In [6]:
import inspect
import types


def execute_source(callback_imports, callback_name, callback_source, args):
    for callback_import in callback_imports:
        exec(callback_import, globals())
    exec('import time' + "\n" + callback_source)
    callback = locals()[callback_name]
    return callback(*args)


def submit(executor, callback, *args):
    callback_source = inspect.getsource(callback)
    callback_imports = list(imports(callback.__globals__))
    callback_name = callback.__name__
    future = executor.submit(
        execute_source,
        callback_imports, callback_name, callback_source, args
    )
    return future


def imports(callback_globals):
    for name, val in list(callback_globals.items()):
        if isinstance(val, types.ModuleType) and val.__name__ != 'builtins' and val.__name__ != __name__:
            import_line = 'import ' + val.__name__
            if val.__name__ != name:
                import_line += ' as ' + name
            yield import_line

In [7]:
# from concurrent.futures import ProcessPoolExecutor
# def multiprocessing(func, args, workers):
#     with ProcessPoolExecutor(max_workers=workers) as executor:
#         res = executor.map(func, args)
#     return list(res)

In [8]:
from concurrent.futures import as_completed, ProcessPoolExecutor
import time
import numpy as np

def do_work(idx1, idx2):
    time.sleep(0.2)
    return np.mean([idx1, idx2])

with ProcessPoolExecutor(max_workers=4) as executor:
    futures = set()
    for idx in range(32):
        future = submit(executor, do_work, idx, idx * 2)
        futures.add(future)

    for future in as_completed(futures):
        print(future.result())

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\ProgramData\Anaconda3\lib\concurrent\futures\process.py", line 295, in _queue_management_worker
    shutdown_worker()
  File "C:\ProgramData\Anaconda3\lib\concurrent\futures\process.py", line 253, in shutdown_worker
    call_queue.put_nowait(None)
  File "C:\ProgramData\Anaconda3\lib\multiprocessing\queues.py", line 129, in put_nowait
    return self.put(obj, False)
  File "C:\ProgramData\Anaconda3\lib\multiprocessing\queues.py", line 83, in put
    raise Full
queue.Full



BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [12]:
%time multiprocessing(make_f, seq_list, 2)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.